# Projection of 3D geometries into Woodscape images

This sample reads Woodscape extrinsic information (poses), just like in [03_Poses_from_quaternions](./03_Poses_from_quaternions.ipynb), but also the intrincs (fisheye) to project data into the images.

Let's use VCD routines to store extrinsics and intrinsics, and test the projection of some 3D entities into the images.

In [ ]:
import numpy as np
import json
import cv2
from matplotlib import pyplot as plt
from PIL import Image

from vcd import utils, core, types, scl, draw


def q2R(q):
    # e.g. rotation_fv = q2R(np.array(quaternion_fv).reshape(4,1))
    # Equivalent to:
    # rotation_fv = SciRot.from_quat(quaternion_fv).as_matrix()

    assert(q.shape == (4, 1))
    x = q[0, 0]
    y = q[1, 0]
    z = q[2, 0]
    w = q[3, 0]
    
    x2 = x*x
    y2 = y*y
    z2 = z*z
    xy = x*y
    xz = x*z
    yz = y*z
    xw = x*w
    yw = y*w
    zw = z*w

    R1 = np.array([[1-2*y2-2*z2, 2*xy-2*zw, 2*xz+2*yw],
                  [2*xy+2*zw, 1-2*x2-2*z2, 2*yz-2*xw],
                  [2*xz-2*yw, 2*yz+2*xw, 1-2*x2-2*y2]])
    return R1


def plot_cs(pose_wrt_ref, name, L=1):
    # Explore the coordinate systems defined for this scene
    axis = np.array([[0, L, 0, 0, 0, 0],
            [0, 0, 0, L, 0, 0],
            [0, 0, 0, 0, 0, L],
            [1, 1, 1, 1, 1, 1]])  # matrix with several 4x1 points
    pose_wrt_ref = np.array(pose_wrt_ref).reshape(4, 4)
    axis_ref = pose_wrt_ref.dot(axis)
    origin = axis_ref[:, 0]
    x_axis_end = axis_ref[:, 1]
    y_axis_end = axis_ref[:, 3]
    z_axis_end = axis_ref[:, 5]
    ax.plot([origin[0], x_axis_end[0]], [origin[1], x_axis_end[1]], [origin[2], x_axis_end[2]], 'r-')
    ax.plot([origin[0], y_axis_end[0]], [origin[1], y_axis_end[1]], [origin[2], y_axis_end[2]], 'g-')
    ax.plot([origin[0], z_axis_end[0]], [origin[1], z_axis_end[1]], [origin[2], z_axis_end[2]], 'b-')

    ax.text(origin[0], origin[1], origin[2], r'{}'.format(name))
    ax.text(x_axis_end[0], x_axis_end[1], x_axis_end[2], 'X')
    ax.text(y_axis_end[0], y_axis_end[1], y_axis_end[2], 'Y')
    ax.text(z_axis_end[0], z_axis_end[1], z_axis_end[2], 'Z')


Let's read 4 sample images and calibrations from Woodscape

In [ ]:
# Read file: intrinsics, extrinsics
with open('../data/Woodscape/00000_FV.json') as f:
    config_fv = json.load(f)

intrinsic_fv = config_fv['intrinsic']
coefficients_fv = [intrinsic_fv['k1'], intrinsic_fv['k2'], intrinsic_fv['k3'], intrinsic_fv['k4']]
traslation_fv = config_fv['extrinsic']['translation']
quaternion_fv = config_fv['extrinsic']['quaternion']
rotation_fv = q2R(np.array(quaternion_fv).reshape(4,1))
R_fv_wrt_lcs = np.array(rotation_fv)
C_fv_wrt_lcs = np.transpose(np.array([traslation_fv]))
P_fv_wrt_lcs = utils.create_pose(R_fv_wrt_lcs, C_fv_wrt_lcs)

with open('../data/Woodscape/00086_RV.json') as f:
    config_rv = json.load(f)
intrinsic_rv = config_rv['intrinsic']
coefficients_rv = [intrinsic_rv['k1'], intrinsic_rv['k2'], intrinsic_rv['k3'], intrinsic_rv['k4']]
traslation_rv = config_rv['extrinsic']['translation']
quaternion_rv = config_rv['extrinsic']['quaternion']
rotation_rv = q2R(np.array(quaternion_rv).reshape(4,1))
R_rv_wrt_lcs = np.array(rotation_rv)
C_rv_wrt_lcs = np.transpose(np.array([traslation_rv]))
P_rv_wrt_lcs = utils.create_pose(R_rv_wrt_lcs, C_rv_wrt_lcs)

with open('../data/Woodscape/00141_MVL.json') as f:
    config_mvl = json.load(f)
intrinsic_mvl = config_mvl['intrinsic']
coefficients_mvl = [intrinsic_mvl['k1'], intrinsic_mvl['k2'], intrinsic_mvl['k3'], intrinsic_mvl['k4']]
traslation_mvl = config_mvl['extrinsic']['translation']
quaternion_mvl = config_mvl['extrinsic']['quaternion']
rotation_mvl = q2R(np.array(quaternion_mvl).reshape(4,1))
R_mvl_wrt_lcs = np.array(rotation_mvl)
C_mvl_wrt_lcs = np.transpose(np.array([traslation_mvl]))
P_mvl_wrt_lcs = utils.create_pose(R_mvl_wrt_lcs, C_mvl_wrt_lcs)

with open('../data/Woodscape/00096_MVR.json') as f:
    config_mvr = json.load(f)
intrinsic_mvr = config_mvr['intrinsic']
coefficients_mvr = [intrinsic_mvr['k1'], intrinsic_mvr['k2'], intrinsic_mvr['k3'], intrinsic_mvr['k4']]
traslation_mvr = config_mvr['extrinsic']['translation']
quaternion_mvr = config_mvr['extrinsic']['quaternion']
rotation_mvr = q2R(np.array(quaternion_mvr).reshape(4,1))
R_mvr_wrt_lcs = np.array(rotation_mvr)
C_mvr_wrt_lcs = np.transpose(np.array([traslation_mvr]))
P_mvr_wrt_lcs = utils.create_pose(R_mvr_wrt_lcs, C_mvr_wrt_lcs)

Now, let's create the VCD and create the cameras

In [ ]:
# Create VCD object
vcd = core.VCD()

# Add parent coordinate system
vcd.add_coordinate_system("vehicle-iso8855", cs_type=types.CoordinateSystemType.local_cs)

# Add streams (cameras)
vcd.add_stream(stream_name="FV", uri="", description="Frontal camera", stream_type=core.StreamType.camera)
vcd.add_stream(stream_name="RV", uri="", description="Rear camera", stream_type=core.StreamType.camera)
vcd.add_stream(stream_name="MVL", uri="", description="Mirror left camera", stream_type=core.StreamType.camera)
vcd.add_stream(stream_name="MVR", uri="", description="Mirro right camera", stream_type=core.StreamType.camera)

# Add extrinsics
vcd.add_coordinate_system('FV', cs_type=types.CoordinateSystemType.sensor_cs,
                         parent_name="vehicle-iso8855",
                         pose_wrt_parent=types.PoseData(
                             val=list(P_fv_wrt_lcs.flatten()),
                             type=types.TransformDataType.matrix_4x4)
                         )
vcd.add_coordinate_system('RV', cs_type=types.CoordinateSystemType.sensor_cs,
                         parent_name="vehicle-iso8855",
                         pose_wrt_parent=types.PoseData(
                             val=list(P_rv_wrt_lcs.flatten()),
                             type=types.TransformDataType.matrix_4x4)
                         )
vcd.add_coordinate_system('MVL', cs_type=types.CoordinateSystemType.sensor_cs,
                         parent_name="vehicle-iso8855",
                         pose_wrt_parent=types.PoseData(
                             val=list(P_mvl_wrt_lcs.flatten()),
                             type=types.TransformDataType.matrix_4x4)
                         )
vcd.add_coordinate_system('MVR', cs_type=types.CoordinateSystemType.sensor_cs,
                         parent_name="vehicle-iso8855",
                         pose_wrt_parent=types.PoseData(
                             val=list(P_mvr_wrt_lcs.flatten()),
                             type=types.TransformDataType.matrix_4x4)
                         )

# Add intrinsics
vcd.add_stream_properties(stream_name="FV",
                         intrinsics=types.IntrinsicsFisheye(
                             width_px=int(intrinsic_fv['width']),                             
                             height_px=int(intrinsic_fv['height']),
                             lens_coeffs_1x4=coefficients_fv,
                             center_x=intrinsic_fv['cx_offset'],
                             center_y=intrinsic_fv['cy_offset'],
                             aspect_ratio=1.0
                             )
                         )
vcd.add_stream_properties(stream_name="RV",
                         intrinsics=types.IntrinsicsFisheye(
                             width_px=int(intrinsic_rv['width']),
                             height_px=int(intrinsic_rv['height']),
                             lens_coeffs_1x4=coefficients_rv,
                             center_x=intrinsic_rv['cx_offset'],
                             center_y=intrinsic_rv['cy_offset'],
                             aspect_ratio=1.0
                             )
                         )
vcd.add_stream_properties(stream_name="MVL",
                         intrinsics=types.IntrinsicsFisheye(
                             width_px=int(intrinsic_mvl['width']),
                             height_px=int(intrinsic_mvl['height']),
                             lens_coeffs_1x4=coefficients_mvl,
                             center_x=intrinsic_mvl['cx_offset'],
                             center_y=intrinsic_mvl['cy_offset'],
                             aspect_ratio=1.0
                             )
                         )
vcd.add_stream_properties(stream_name="MVR",
                         intrinsics=types.IntrinsicsFisheye(
                             width_px=int(intrinsic_mvr['width']),
                             height_px=int(intrinsic_mvr['height']),
                             lens_coeffs_1x4=coefficients_mvr,
                             center_x=intrinsic_mvr['cx_offset'],
                             center_y=intrinsic_mvr['cy_offset'],
                             aspect_ratio=1.0
                             )
                         )

Now, let's create some 3D objects

In [ ]:
def add_some_objects(vcd):
    #########################################
    # Cuboids
    #########################################
    uid1 = vcd.add_object(name="", semantic_type="Car", frame_value=0)
    cuboid1 = types.cuboid(name="box3D",
                           val=(4.5, 3.5, 0.7,
                                0.0, 0.0, (90.0 * np.pi) / 180.0,
                                4.2, 1.8, 1.4),
                           coordinate_system="vehicle-iso8855")
    vcd.add_object_data(uid1, cuboid1, frame_value=0)

    #########################################
    # Points3d (Walls)
    #########################################
    xm, ym, zm = utils.generate_grid(x_params=(0, 20, 21), y_params=(-20, 20, 41), z_params=(0, 0, 1))
    points3d_4xN = utils.grid_as_4xN_points3d(xm, ym, zm)
    uid_wall_1 = vcd.add_object(name="ground_x_pos", semantic_type="Ground", frame_value=0)
    mat_wall = types.mat(name="wall",
                         val=points3d_4xN.flatten().tolist(),
                         channels=1,
                         width=points3d_4xN.shape[1],
                         height=points3d_4xN.shape[0],
                         dataType='float',
                         coordinate_system="vehicle-iso8855")
    # mat_wall.add_attribute(types.vec(name="color",
    #                                 val=(255, 0, 0)))
    vcd.add_object_data(uid_wall_1, mat_wall, frame_value=0)

    xm, ym, zm = utils.generate_grid(x_params=(0, -20, 21), y_params=(-20, 20, 41), z_params=(0, 0, 1))
    points3d_4xN = utils.grid_as_4xN_points3d(xm, ym, zm)
    uid_wall_2 = vcd.add_object(name="ground_x_neg", semantic_type="Ground", frame_value=0)
    mat_wall = types.mat(name="wall",
                         val=points3d_4xN.flatten().tolist(),
                         channels=1,
                         width=points3d_4xN.shape[1],
                         height=points3d_4xN.shape[0],
                         dataType='float',
                         coordinate_system="vehicle-iso8855")
    # mat_wall.add_attribute(types.vec(name="color",
    #                                val=(255, 255, 0)))
    vcd.add_object_data(uid_wall_2, mat_wall, frame_value=0)

    xm, ym, zm = utils.generate_grid(x_params=(20, 20, 1), y_params=(-20, 20, 41), z_params=(0, 2, 21))
    points3d_4xN = utils.grid_as_4xN_points3d(xm, ym, zm)
    uid_wall_3 = vcd.add_object(name="wall_front", semantic_type="Wall", frame_value=0)
    mat_wall = types.mat(name="wall",
                         val=points3d_4xN.flatten().tolist(),
                         channels=1,
                         width=points3d_4xN.shape[1],
                         height=points3d_4xN.shape[0],
                         dataType='float',
                         coordinate_system="vehicle-iso8855")
    # mat_wall.add_attribute(types.vec(name="color",
    #                                 val=(0, 255, 0)))
    vcd.add_object_data(uid_wall_3, mat_wall, frame_value=0)

    xm, ym, zm = utils.generate_grid(x_params=(-20, -20, 1), y_params=(-20, 20, 41), z_params=(0, 2, 21))
    points3d_4xN = utils.grid_as_4xN_points3d(xm, ym, zm)
    uid_wall_4 = vcd.add_object(name="wall_rear", semantic_type="Wall", frame_value=0)
    mat_wall = types.mat(name="wall",
                         val=points3d_4xN.flatten().tolist(),
                         channels=1,
                         width=points3d_4xN.shape[1],
                         height=points3d_4xN.shape[0],
                         dataType='float',
                         coordinate_system="vehicle-iso8855")
    # mat_wall.add_attribute(types.vec(name="color",
    #                                 val=(0, 255, 255)))
    vcd.add_object_data(uid_wall_4, mat_wall, frame_value=0)

    xm, ym, zm = utils.generate_grid(x_params=(-20, 20, 41), y_params=(20, 20, 1), z_params=(0, 2, 21))
    points3d_4xN = utils.grid_as_4xN_points3d(xm, ym, zm)
    uid_wall_5 = vcd.add_object(name="wall_right", semantic_type="Wall", frame_value=0)
    mat_wall = types.mat(name="wall",
                         val=points3d_4xN.flatten().tolist(),
                         channels=1,
                         width=points3d_4xN.shape[1],
                         height=points3d_4xN.shape[0],
                         dataType='float',
                         coordinate_system="vehicle-iso8855")
    # mat_wall.add_attribute(types.vec(name="color",
    #                                 val=(255, 0, 255)))
    vcd.add_object_data(uid_wall_5, mat_wall, frame_value=0)

    xm, ym, zm = utils.generate_grid(x_params=(-20, 20, 41), y_params=(-20, -20, 1), z_params=(0, 2, 21))
    points3d_4xN = utils.grid_as_4xN_points3d(xm, ym, zm)
    uid_wall_6 = vcd.add_object(name="wall_left", semantic_type="Wall", frame_value=0)
    mat_wall = types.mat(name="wall",
                         val=points3d_4xN.flatten().tolist(),
                         channels=1,
                         width=points3d_4xN.shape[1],
                         height=points3d_4xN.shape[0],
                         dataType='float',
                         coordinate_system="vehicle-iso8855")
    # mat_wall.add_attribute(types.vec(name="color",
    #                                 val=(255, 255, 0)))
    vcd.add_object_data(uid_wall_6, mat_wall, frame_value=0)

    #########################################
    # Ego-vehicle
    #########################################
    vcd.add_object(name="Ego-car", semantic_type="Ego-car", uid=str(-2))

    cuboid_ego = types.cuboid(name="box3D",
                              val=(1.35, 0.0, 0.736,
                                   0.0, 0.0, 0.0,
                                   4.765, 1.82, 1.47),
                              coordinate_system="vehicle-iso8855")
    vcd.add_object_data(str(-2), cuboid_ego)

    return vcd

In [ ]:
vcd = add_some_objects(vcd)

And finally draw the scene.

In [ ]:
# Create scene object, which reads VCD and has functions to convert data
scene = scl.Scene(vcd)

# Create image drawers
drawer_fv = draw.Image(scene, "FV")
drawer_rv = draw.Image(scene, "RV")
drawer_mvl = draw.Image(scene, "MVL")
drawer_mvr = draw.Image(scene, "MVR")

# Create also the set-up viewer
setupViewer = draw.SetupViewer(scene, "vehicle-iso8855")
fig1 = setupViewer.plot_setup()
plt.show()

'''
# Visualize
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

# Let's plot the Vehicle coordinate system
P_lcs_wrt_lcs = np.array([[1.0, 0.0, 0.0, 0.0],[0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0]]) 
plot_cs(P_lcs_wrt_lcs, 'lcs', L=2)
plot_cs(P_fv_wrt_lcs, 'fv', L=1)
plot_cs(P_rv_wrt_lcs, 'rv', L=1)
plot_cs(P_mvl_wrt_lcs, 'mvl', L=1)
plot_cs(P_mvr_wrt_lcs, 'mvr', L=1)

ax.set_xlim(-3, 3)
ax.set_ylim(-3, 3)
ax.set_zlim(0, 6)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
'''

# Read the images
img_fv = cv2.imread('../data/Woodscape/00000_FV.png')
img_rv = cv2.imread('../data/Woodscape/00086_RV.png')
img_mvl = cv2.imread('../data/Woodscape/00141_MVL.png')
img_mvr = cv2.imread('../data/Woodscape/00096_MVR.png')

# Draw
drawer_fv.draw(_img=img_fv, _frameNum=0)
drawer_rv.draw(_img=img_rv, _frameNum=0)
drawer_mvl.draw(_img=img_mvl, _frameNum=0)
drawer_mvr.draw(_img=img_mvr, _frameNum=0)

In [ ]:
Image.fromarray(cv2.cvtColor(img_fv, cv2.COLOR_BGR2RGB))  # .show() to open in external tool

In [ ]:
Image.fromarray(cv2.cvtColor(img_rv, cv2.COLOR_BGR2RGB))

In [ ]:
Image.fromarray(cv2.cvtColor(img_mvl, cv2.COLOR_BGR2RGB))

In [ ]:
Image.fromarray(cv2.cvtColor(img_mvr, cv2.COLOR_BGR2RGB))

Let's save the VCD JSON file for further utilisation.

In [ ]:
vcd.save("../data/vcd431_woodscape_samples.json")